# Foundations of Statistical Analysis and Machine Learning - Final exam exercise 2/2

### <span style="color:red"> WRITE YOUR NAME HERE: </span>   

### Guidelines

The exam is composed of two exercises (on separate notebooks) each one with a specific dataset. In total, there are three independent parts to complete:
* Exercise 1 (8 points): regression
* Exercise 2 part 1 (9 points): classification
* Exercise 2 part 2 (3 points): clustering

You can work directly on the notebooks. At the end of the 2.5-hour exam, you will have 5 minutes to upload it on Teams.

Don't forget that you have to complete the TWO notebooks to have a passing grade.

Many questions can be tackled even if the previous ones are not completed or not correct.

The exam is long on purpose, so don't worry if you cannot complete 100 % of the questions. Do as much as you can. In case you are blocked at some point, don't panic. Just move on to the next question. And remember written answers where you have to explain or comment your results are worth many points.

Avoid "naive" copy-pasting: you will not understand what you are doing and it will be a problem for the next questions. Notebooks that are cluttered with useless code that has been mindlessly copied will be penalized. Just do what is asked at each question. 

Moreover, you are reminded that any messaging with each other during the exam or use of generative AI models such as ChatGPT, Bard or Bing will be considered as cheating and will result in the termination of your exam. Moreover, similarities between student works are easy to spot. 

If a question seems unclear and you are not sure about what you have to do, do not hesitate to raise your hand and ask for clarification.

Good luck!

## Exercise 2

Here are some libraries that could be usefull in the exercises.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn 

We will work on the provided data set BankChurners that gathers information on the customers of a bank.

In [2]:
# Loading the original data set
data = pd.read_csv("BankChurners.csv")
data.head(10)

CLIENTNUM     Attrition_Flag  Customer_Age Gender  Dependent_count  \
0  768805383  Existing Customer            45      M                3   
1  818770008  Existing Customer            49      F                5   
2  713982108  Existing Customer            51      M                3   
3  769911858  Existing Customer            40      F                4   
4  709106358  Existing Customer            40      M                3   
5  713061558  Existing Customer            44      M                2   
6  810347208  Existing Customer            51      M                4   
7  818906208  Existing Customer            32      M                0   
8  710930508  Existing Customer            37      M                3   
9  719661558  Existing Customer            48      M                2   

  Education_Level Marital_Status Income_Category Card_Category  \
0     High School        Married     $60K - $80K          Blue   
1        Graduate         Single  Less than $40K          Blue   
2        Graduate        Married    $80K - $120K          Blue   
3     High School        Unknown  Less than $40K          Blue   
4      Uneducated        Married     $60K - $80K          Blue   
5        Graduate        Married     $40K - $60K          Blue   
6         Unknown        Married         $120K +          Gold   
7     High School        Unknown     $60K - $80K        Silver   
8      Uneducated         Single     $60K - $80K          Blue   
9        Graduate         Single    $80K - $120K          Blue   

   Months_on_book  ...  Credit_Limit  Total_Revolving_Bal  Avg_Open_To_Buy  \
0              39  ...       12691.0                  777          11914.0   
1              44  ...        8256.0                  864           7392.0   
2              36  ...        3418.0                    0           3418.0   
3              34  ...        3313.0                 2517            796.0   
4              21  ...        4716.0                    0           4716.0   
5              36  ...        4010.0                 1247           2763.0   
6              46  ...       34516.0                 2264          32252.0   
7              27  ...       29081.0                 1396          27685.0   
8              36  ...       22352.0                 2517          19835.0   
9              36  ...       11656.0                 1677           9979.0   

   Total_Amt_Chng_Q4_Q1  Total_Trans_Amt  Total_Trans_Ct  Total_Ct_Chng_Q4_Q1  \
0                 1.335             1144              42                1.625   
1                 1.541             1291              33                3.714   
2                 2.594             1887              20                2.333   
3                 1.405             1171              20                2.333   
4                 2.175              816              28                2.500   
5                 1.376             1088              24                0.846   
6                 1.975             1330              31                0.722   
7                 2.204             1538              36                0.714   
8                 3.355             1350              24                1.182   
9                 1.524             1441              32                0.882   

   Avg_Utilization_Ratio  \
0                  0.061   
1                  0.105   
2                  0.000   
3                  0.760   
4                  0.000   
5                  0.311   
6                  0.066   
7                  0.048   
8                  0.113   
9                  0.144   

   Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1  \
0                                           0.000093                                                                                    
1                                           0.000057                                                                                    
2

Here is some code to execute to prepare the data set.

In [3]:
# Removing the last two columns that are useless
data = data.iloc[:,:-2]

# Encoding the customer churn
data.Attrition_Flag = data.Attrition_Flag.replace({'Attrited Customer':1,'Existing Customer':0})

# Encoding the gender
data.Gender = data.Gender.replace({'F':1,'M':0})

# Dropping rows with unknown values
data = data.drop(data[data.Education_Level=='Unknown'].index, axis=0)
data = data.drop(data[data.Income_Category=='Unknown'].index, axis=0)

# Encoding the education level with an (ordered) scale
data.Education_Level = data.Education_Level.replace({'Uneducated':0,'High School':1,'College':2,'Graduate':3,'Post-Graduate':4,'Doctorate':5})

# Encoding the income category with the mean value of each income interval
data.Income_Category = data.Income_Category.replace({'Less than $40K':20,'$40K - $60K':50,'$60K - $80K':70,'$80K - $120K':100,'$120K +':150})

# Encoding the marital status
data = pd.concat([data,pd.get_dummies(data['Marital_Status']).drop(columns=['Unknown'])],axis=1)

# Encoding the card category
data = pd.concat([data,pd.get_dummies(data['Card_Category'], prefix='Card')], axis=1)
# data = pd.concat([data,pd.get_dummies(data['Card_Category']).drop(columns=['Platinum'])],axis=1)
# data.drop(columns = ['Education_Level','Income_Category','Marital_Status','Card_Category','CLIENTNUM'],inplace=True)

# Removing the useless columns
data = data.drop(columns = ['Marital_Status','Card_Category','CLIENTNUM'])

data.head(10)

Attrition_Flag  Customer_Age  Gender  Dependent_count  Education_Level  \
0                0            45       0                3                1   
1                0            49       1                5                3   
2                0            51       0                3                3   
3                0            40       1                4                1   
4                0            40       0                3                0   
5                0            44       0                2                3   
7                0            32       0                0                1   
8                0            37       0                3                0   
9                0            48       0                2                3   
10               0            42       0                5                0   

    Income_Category  Months_on_book  Total_Relationship_Count  \
0                70              39                         5   
1                20              44                         6   
2               100              36                         4   
3                20              34                         3   
4                70              21                         5   
5                50              36                         3   
7                70              27                         2   
8                70              36                         5   
9               100              36                         6   
10              150              31                         5   

    Months_Inactive_12_mon  Contacts_Count_12_mon  ...  Total_Trans_Ct  \
0                        1                      3  ...              42   
1                        1                      2  ...              33   
2                        1                      0  ...              20   
3                        4                      1  ...              20   
4                        1                      0  ...              28   
5                        1                      2  ...              24   
7                        2                      2  ...              36   
8                        2                      0  ...              24   
9                        3                      3  ...              32   
10                       3                      2  ...              42   

    Total_Ct_Chng_Q4_Q1  Avg_Utilization_Ratio  Divorced  Married  Single  \
0                 1.625                  0.061         0        1       0   
1                 3.714                  0.105         0        0       1   
2                 2.333                  0.000         0        1       0   
3                 2.333                  0.760         0        0       0   
4                 2.500                  0.000         0        1       0   
5                 0.846                  0.311         0        1       0   
7                 0.714                  0.048         0        0       0   
8                 1.182                  0.113         0        0       1   
9                 0.882                  0.144         0        0       1   
10                0.680                  0.217         0        0       0   

    Card_Blue  Card_Gold  Card_Platinum  Card_Silver  
0           1          0              0            0  
1           1          0              0            0  
2           1          0              0            0  
3           1          0              0            0  
4           1          0              0            0  
5           1          0              0            0  
7           0          0              0            1  
8           1          0              0            0  
9           1          0              0            0  
10          1          0              0            0  

[10 rows x 25 columns]

You can assume that the data set is cleaned and prepared now.

## PART 1: Predicting the churn (9 points)

The director of EPIbank has a problem. He is losing a lot of clients who prefer to go open a bank account with the main competitor, BNP Paribank. 

He asks you to develop a model that recognizes clients that are likely to leave EPIbank (i.e. to churn) and go somewhere else. Your model will be used to spot the clients to whom he could offer special benefits to retain them before they leave.

For this task, you have the dataset shown above, with data about past customers, including if they left EPIbank or not.  
**Attrition_Flag** corresponds to the customer churn and is our target variable here. The value is 0 when the customer is still in the bank, and is 1 when she has left the bank and is no longer a customer.  
The other columns will be considered as predictors (or features).

### 1) Prepare y (for the target) and X (for the predictors)

### 2) Make density plots to compare the distributions of the following three features depending on the Attrition_Flag: Customer_Age, Total_Trans_Ct, Total_Revolving_Bal
In other words:
- On the first plot: show the distribution of Customer_Age with one color for the customers who have left (Attrition_Flag = 1) and another color for the customers who are still in the bank (Attrition_Flag = 0). 
- Plot the same for Total_Trans_Ct and Total_Revolving_Bal 
- (3 plots in total)

### 3) Plot the repartition of Gender (men and women) among customers who have left (Attrition_Flag = 1) and customers who are still in the bank (Attrition_Flag = 0).  

### 4) When you look at the figures of questions 2 and 3, which predictor(s) do you think is (are) the best for predicting the churn among the following?: (Explain why)
- Customer_Age 
- Total_Trans_Ct 
- Total_Revolving_Bal
- Gender

### 5) Proceed to a split of the data set. Bear in mind that you will be asked for an accurate estimation of the performance of your best model at the end. Keep 60 % of the examples for the training set.

### 6) Choose ONE metric that you will use to evaluate models that will predict churn. What metric do you use? Explain why you choose this metric, based on the use case.

### 7) Train a model of your choice (except Random Forest), using all features, and tune AT LEAST ONE hyperparameter.
Note: Don't forget to re-train your model with the optimal hyperparameter you found.

### 8) Train a Random Forest, using all features, and tune the number of estimators
Note: Don't forget to re-train your RandomForest with the optimal hyperparameter you found.

### 9) Compare the performance of the models you have trained. Comment the results based on the metric you chose earlier.

### 10) Plot the ROC curves for your models. Does it confirm your choice? Explain.

## PART 2: Making clusters of customers (3 points)

In this part, we will not consider the churn, we will on focus on the information in X to create three clusters of customers. Customers among each cluster should share similarities so that specific employees can be attributed to each cluster.

### 11) Train a k-Means clustering with k = 3 using the following features:
- Credit_Limit
- Income_Category

### 12) On the feature plane made of _Plot Credit_Limit_ vs. _Income_Category_, plot the customers as points with colors corresponding to the cluster attribution

### 13) Describe the clusters that you have formed. In other words, what can you say are the characteristics of customers that belong to each of the clusters (0, 1 and 2)?